The MIT License (MIT)

Copyright (c) 2021 NVIDIA CORPORATION

Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
fnames = ['cpmp_0' + str(i) for i in range(89, 98) if i != 95]
fnames

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
from tqdm import tqdm
from scipy.special import expit, logit
pd.options.display.max_columns = 200

In [ ]:
oof = {}
for fname in tqdm(fnames):
    with open ('./checkpoints/%s/%s_oof.pkl' % (fname, fname), 'rb') as file:
        oof[fname] = pkl.load(file)


In [ ]:
for fname in fnames:
    print(fname, oof[fname]['city_map'].shape)

In [ ]:
def get_topN(preds, N):
    TOPN = np.empty((preds.shape[0], N))
    PREDN = np.empty((preds.shape[0], N))
    preds = preds.copy()
    for i in tqdm(range(N)):
        x = np.argmax(preds, axis=1)
        TOPN[:,i] = x
        x = np.expand_dims(x, axis=1)
        PREDN[:,i] = np.take_along_axis(preds, x, axis=1).ravel()
        np.put_along_axis(preds, x, -1e10, axis=1)
    return TOPN, PREDN


In [ ]:
CITY_MAP = oof[fname]['city_map'].to_pandas().to_dict()

In [ ]:
np.vectorize(CITY_MAP.get)(top_preds)

In [ ]:
valid_cities =  oof[fname]['valid_cities']

In [ ]:
utrip_id =  oof[fname]['valid_trips'].ravel()
utrip_id

In [ ]:
for fname in fnames:
    train_preds = oof[fname]['preds_all']
    top_preds, top_logits = get_topN(train_preds.copy(), 4)
    top_preds = top_preds[:, :11158].astype('int')
    top_preds = np.vectorize(CITY_MAP.get)(top_preds)
    score = np.mean(np.max(valid_cities == top_preds, axis=1))
    print(fname, score)

In [ ]:
def my_softmax(x):
    x = x - x.max(1, keepdims=True)
    x = np.exp(x)
    x = x / x.sum(1, keepdims=True)
    return x

In [ ]:
train_preds = 0
for fname in tqdm(fnames):
    train_preds = train_preds + oof[fname]['preds_all'][:, :11158]
    
train_preds = my_softmax(train_preds / len(fnames))


top_preds, top_logits = get_topN(train_preds.copy(), 4)


top_preds = top_preds.astype('int')

top_preds = np.vectorize(CITY_MAP.get)(top_preds)

top_preds

np.mean(np.max(valid_cities == top_preds, axis=1))

In [ ]:
top_preds, top_logits = get_topN(train_preds.copy(), 50)
top_preds = top_preds.astype('int')

In [ ]:
train_oof = pd.DataFrame({'utrip_id':utrip_id})
train_oof

In [ ]:
for i in tqdm(range(50)):
    colname = 'rec%d' % i
    train_oof[colname] = np.vectorize(CITY_MAP.get)(top_preds[:, i])

In [ ]:
for i in tqdm(range(50)):
    colname = 'prob%d' % i
    train_oof[colname] = top_logits[:, i]

In [ ]:
train_oof

In [ ]:
train_oof.to_csv('./cpmp_train_oof.csv', index=False)

In [ ]:
test_final = pd.read_csv('test_final.csv')
test_final

In [ ]:
test_final = test_final[['utrip_id']].copy()
test_final

In [ ]:
test_preds = 0
for fname in tqdm(fnames):
    test_preds = test_preds + oof[fname]['test_preds_all'][:, :11158]
    
test_preds = my_softmax(5 * test_preds / len(fnames))  # fix bug

top_preds, top_logits = get_topN(test_preds.copy(), 50)
top_preds = top_preds.astype('int')

for i in tqdm(range(50)):
    colname = 'rec%d' % i
    test_final[colname] = np.vectorize(CITY_MAP.get)(top_preds[:, i])
for i in tqdm(range(50)):
    colname = 'prob%d' % i
    test_final[colname] = top_logits[:, i]

In [ ]:
test_final

In [ ]:
test_final.to_csv('./cpmp_test_final_fixed.csv', index=False)

In [ ]:
_ = plt.hist(oof['cpmp_093']['preds_all'][:10000].ravel().clip(-50, 50), bins=100, log=True)

In [ ]:
_ = plt.hist(oof['cpmp_097']['test_preds_all'][:10000].ravel().clip(-10, 10)*5, bins=100)